In [2]:
pip install tensorflow 

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using ca

In [3]:
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [6]:
# Load the dataset
with open("C:\\Users\\jaini\\Downloads\\dataset.txt", 'r') as file:
    names = file.readlines()

In [7]:
# Preprocessing
names = ['^' + name.strip().lower() + '$' for name in names]  # Add start and end of sequence tokens

In [8]:
# Get the unique characters in the dataset
chars = sorted(list(set(''.join(names))))

In [9]:
# Creating a mapping from characters to integers and vice versa
char_to_int = {char: i for i, char in enumerate(chars)}
int_to_char = {i: char for i, char in enumerate(chars)}

In [10]:
# Creating sequences for training
max_len = max([len(name) for name in names])
sequences = []
next_chars = []
for name in names:
    for i in range(len(name) - 1):
        sequences.append(name[:i+1])
        next_chars.append(name[i+1])

In [12]:
# Vectorizing sequences
X = np.zeros((len(sequences), max_len, len(chars)), dtype=bool)
y = np.zeros((len(sequences), len(chars)), dtype=bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

In [13]:
# Build the model
model = Sequential([
    LSTM(128, input_shape=(max_len, len(chars))),
    Dropout(0.2),
    Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# Train the model
model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50

57060/57060 [==============================] - 1939s 34ms/step - loss: 1.6373
Epoch 2/50
57060/57060 [==============================] - 1946s 34ms/step - loss: 1.4272
Epoch 3/50
57060/57060 [==============================] - 2109s 37ms/step - loss: 1.3944
Epoch 4/50
57060/57060 [==============================] - 2118s 37ms/step - loss: 1.3800
Epoch 5/50
57060/57060 [==============================] - 1729s 30ms/step - loss: 1.3694
Epoch 6/50
57060/57060 [==============================] - 1383s 24ms/step - loss: 1.3621
Epoch 7/50
57060/57060 [==============================] - 1609s 28ms/step - loss: 1.3570
Epoch 8/50
57060/57060 [==============================] - 2124s 37ms/step - loss: 1.3535
Epoch 9/50
57060/57060 [==============================] - 2149s 38ms/step - loss: 1.3501
Epoch 10/50
57060/57060 [==============================] - 2133s 37ms/step - loss: 1.3474
Epoch 11/50
57060/57060 [==============================] - 2128s 37ms/step - loss: 1.3459
Epoch 12/50
57060/

In [15]:
# Function to generate names
def generate_name(model, seed=None, temperature=1.0, max_len=20):
    if seed is None:
        seed = '^'
    generated = seed
    while len(generated) < max_len:
        x_pred = np.zeros((1, max_len, len(chars)))
        for t, char in enumerate(generated):
            x_pred[0, t, char_to_int[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = int_to_char[next_index]
        if next_char == '$':
            break
        generated += next_char
    return generated

In [16]:
# Helper function to sample an index from a probability array
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
# Streamlit app
st.title('Name Generator')

seed_text = st.text_input('Enter seed text (optional):', '^')
temperature = st.slider('Temperature:', 0.1, 2.0, 1.0, 0.1)
num_names = st.slider('Number of names to generate:', 1, 10, 1)

if st.button('Generate Names'):
    for _ in range(num_names):
        generated_name = generate_name(model, seed=seed_text, temperature=temperature)
        st.write(generated_name)

2024-02-13 17:03:32.804 
  command:

    streamlit run D:\Path\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
